In [1]:
import matplotlib.pyplot as plt
from scipy.integrate import quad
import numpy as np
import scipy
import random
import pandas as pd
import random
from tqdm import tqdm
import re
import os
from scipy.stats import norm
import psrqpy
import seaborn as sns

### function that quantifies similarity between sequences
def simfinder(list1,list2):
    counter=0
    sequ=[]
    for i in range(len(list1)):
        if i == (len(list1)-1) and list1[i]==list2[i]:
            counter+=1
            sequ.append(counter)
            continue
        ###this allows the final point in the sequence to be counted if the same
        elif list1[i]==list2[i] and list1[i+1]==list2[i+1] and counter ==0:
            counter+=2
            ### this identifies where a string of common sequence may begin
        elif list1[i]==list2[i] and list1[i+1]==list2[i+1] and counter !=0:
            counter+=1
            ### allows the quantifiable number of similarity to increase
        else:
            sequ.append(counter)
            counter=0
            ### resets the largest common string counter if the string breaks and is no longer the same
    if len(sequ)==1:
        sequ[0]=sequ[0]-1


    lsc=max(sequ)/len(list1)
    #cosv=scipy.spatial.distance.cosine(list1,list2)
    #ham= scipy.spatial.distance.hamming(list1,list2)

    #return (lsc,cosv,ham)
    return(lsc)

    #return max(sequ)

### takes the average length of the largest common string between the generated sequence and each sequence
### in the background as the simalarity value
def total_sim(mainlist,totlists):
    listlength=len(mainlist)
    number=len(totlists)
    total=0
    maxsim=0
    for i in totlists:
        #lsc,ham,cosv=simfinder(mainlist,i)
        lsc=simfinder(mainlist,i)
        #normlsc=lsc/len(mainlist)
        #normham= (len(mainlist)-ham)/len(mainlist)

        #intsim=(normlsc+normham+cosv)/3
        #intsim=(0.7*normlsc+0.2*normham+0.1*cosv)
        intsim=lsc
        total+=intsim

    total_sim=total/len(totlists)

#         total+=simfinder(mainlist,i)

#     average=total/number
#     total_sim=average/listlength
    return total_sim
###power score is the average number of times a 1 or an "on" signal is seen in the generated function
def totalpower(list1):
    return list1.count(1)/len(list1)

### the cost function whihc is minimized is found by ascribing weight to the simalarity and power values and adding them
def costfunc(simweight,powerweight,allsequences,targetsequence):
    cost= (simweight*total_sim(targetsequence,allsequences))+(powerweight*totalpower(targetsequence))
    totcost=(cost*100)
    return(totcost)

def pulsesim(meanflux,sigma,shift,pointquantity):

    # Generate a range of x values
    x = np.linspace(shift - 3*sigma, shift + 3*sigma, pointquantity)
    # Calculate the y values from the normal distribution
    a=meanflux
    b=shift
    c=sigma

    y = a * np.exp(-(x - b)**2 / (2 * c**2))

    return x,y
# Plot the standard curve
def simpulsereader(Center,fluxcutoff,x,y,z,r):
    ### Here the lists are defined where the variables next will be saved.
    pulsenames=[]
    frequencies=[]
    periods =[]
    pulsars=[]
    intense=[]
    times=[]
    interval1=[]
    interval2=[]
    deltas=[]
    coefficient=[]
    ###Below, the function will choose between the two frequencies available in the database given User Input
    if Center == 400:
        Center== 'S400'
    if Center == 1400:
        Center = 'S1400'
    ###Below the function accesses the ATNF database and uses given parameters to create dataframe
    d= psrqpy.QueryATNF(params=["JNAME",'W50',Center, "P0", "F0", "DecJ","DecJD","RaJ","RaJD","GL","GB","ZZ","XX","YY"])
    data=d.catalogue
    database=data[(data[Center] > fluxcutoff) &(data["ZZ"]<=z+r) & (data["ZZ"]>=z-r) & (data["XX"]<=x+r) & (data["XX"]>=x-r) & (data["YY"]<=y+r) & (data["YY"]>=y-r) ]

    ###the resolution of the sequences is arbitarily defined as 1/100th of the smallest period and 
    ###1.5x the largest period. This helps to ensure all pulsars exhibit pulse within range+phaseshift
    timeres= database["P0"].min() /1000
    tottime=database["P0"].max() *1.5

    #reference is used to create an array that helps to frame the generated sequences
    reference=np.arange(0,tottime,timeres)

    for i in range(len(database[Center])):
        ####below the pulsar info is extracted from database for each pulsar
        pulsename= database["JNAME"].iloc[i]
        period= database["P0"].iloc[i]
        frequency= database['F0'].iloc[i]

        sig = database['W50'].iloc[i]/np.sqrt(np.log(2))
        ###pulsesim will generate a simulated pulse profile based on extracted values
        point,pulse= pulsesim(database[Center].iloc[i],sig,random.randint(0,10000),len(reference))

        t= database["P0"].iloc[i]/len(point)

        pulsenames.append(pulsename)
        periods.append(period)
        intense.append(pulse)
        times.append(t)

        delta=t*point

        deltas.append(delta)

###########################################################################################################
        finalp=len(reference) - 1
        t1=[]
        t2=[]
        for i in range(len(pulse)):
            if i ==0:
                current = pulse[i]
                future =pulse[i+1]
####  model saves data points where  flux values of the pulsar are
 ###above the defined cutoff saving where it goes above the cutoff and where it goes below the cutoff
                if current >= fluxcutoff:
                    x=delta[i]
                    t1.append(x)

            elif i ==finalp:
                current = pulse[i]
                past = pulse[i-1]

                if current >=fluxcutoff:
                    x=delta[i]
                    t2.append(x)
            else:
                current= pulse[i]
                previous = pulse[i-1]
                future = pulse[i+1]

                if current >= fluxcutoff and previous < fluxcutoff and future < fluxcutoff:
                    x=delta[i]
                    t1.append(x)
                    t2.append(x)
                elif current >=fluxcutoff and previous <fluxcutoff:
                    x=delta[i]
                    t1.append(x)

                elif current >=fluxcutoff and future < fluxcutoff:
                    x=delta[i]
                    t2.append(x)
            #if t1 ==[] or t2 ==[]:
                #print(intensity)
                #print(t1,t2)
                #raise ValueError("no times were found, likely issue in intensity data")

        if len(t1) != len(t2):
            #print(intensity)
            raise ValueError("t1 and t2 not same length")


### below the generator checks each time resolution against the above the cutoff time values and ascribes a 1
### if the time resolution is between those values or a zero if it is not.
        coef=round(tottime/period)

        reps1=[]
        reps2=[]
        for i in range(1,coef):
            for x in range(len(t1)):

                n=t1[x]+period*i
                f=t2[x]+period*i
                reps1.append(n)
                reps2.append(f)


        t1= t1+reps1
        t2=t2+reps2



        binary=[]

        for x in range(len(t1)):

                for t in reference:

                    if len(t1) ==1:
                        if t >= t1[x] and t<=t2[x]:
                            binary.append(1)
                        else:
                            binary.append(0)
                    else:
                        if x== 0:
                            if t >= t1[x] and t<=t2[x]:
                                binary.append(1)
                            elif t>t2[x]:
                                continue
                            else:
                                binary.append(0)
                        elif x==len(t1)-1:
                            if t<=t2[x-1]:
                                continue
                            elif t>=t1[x] and t<=t2[x]:
                                binary.append(1)
                            elif t == reference[-1]:
                                binary.append(0)
                            else:
                                binary.append(0)
                        else:
                            if t<=t2[x-1]:
                                continue
                            elif t >= t1[x] and t<=t2[x]:
                                binary.append(1)
                            elif t>t2[x]:
                                continue
                            else:
                                binary.append(0)

        ###If the binary sequence never is above threshold i.e no pulse, binary sequence is all 0s
        if binary==[]:
            binary=0*reference
            binary=binary.tolist()

            pulsars.append(binary)
            interval1.append(t1)
            interval2.append(t2)
            coefficient.append(coef)
        else:
            pulsars.append(binary)
            interval1.append(t1)
            interval2.append(t2)
            coefficient.append(coef)

    return( pulsenames,periods, pulsars, intense,interval1,interval2,reference,deltas, times,coefficient)

def mutate(sequence):
    newsequence=[]
    L = len(sequence)
    r = random.randint(0, L-1)
    for i in sequence:
        newsequence.append(i)
    newsequence[r] = 1 - newsequence[r]  # This will flip 0 to 1 and 1 to 0
    return newsequence

def pulsar_model(backgroundpulses='NA'):
    ###user defines the weights of the similarity and energy values
    sim= float(input("Enter Similarity Value Between 0.0 and 1.0: "))
    egy= float(input("Enter Energy Value Between 0.0 and 1.0: "))

    ###if the user does not already have a defined list of background binary pulses, will generate for them
    if backgroundpulses == 'NA':
        Center= int(input("Enter Initial Channel Wavelength: "))
        fluxcutoff= float(input("Enter Minimum Flux To Be Considered A Pulse: "))
        x=float(input("Enter Galactic X coordinate Of Search Area Center: "))
        y=float(input("Enter Galactic Y coordinate Of Search Area Center: "))
        z=float(input("Enter Galactic Z coordinate Of Search Area Center: "))
        r=float(input("Enter Radius of Search Area in Kpc: "))
        
        pulsarnames,periods,ogpulsars,fluxprofile,interval1,interval2,reference,deltas,times,coefficient= simpulsereader(Center,fluxcutoff,x,y,z,r)
        mut=[0]*len(reference)
    else:
        ogpulsars= backgroundpulses
        mut=[0]*len(ogpulsars[0])
    
    ###c is a counter for number of iterations, prev a place holder for previous sequence cost, count 
    ###used as optimization check  
    c=0
    prev=0
    count=0   
    
    fin = 40
    ###below creates a progress bar for model
    with tqdm(total=fin, desc="Processing", unit="iteration") as pbar:
        count= 0
        ###below forces the the model to keep iterating until cost is <0.01% or 4000 iterations of 
        ###no change in cost occur 
        while count < fin:  
            while costfunc(sim,egy,ogpulsars, mut)>0.01:
                muttest=[]
                ###function initially mutates one element from 0-1 or 1-0 
                for i in mut:
                    muttest.append(i)
        
                mut1=mutate(muttest)
                #mut1=mutate(mut1)
                #mut1=mutate(mut1)
                #mut1=mutate(mut1)
                #mut1=mutate(mut1)
                #mut1=mutate(mut1)
                
                ###if mutated sequence has lower cost, save the new sequence as mut
                if costfunc(sim,egy,ogpulsars,mut1) < costfunc(sim,egy,ogpulsars,mut):
                    mut = []
                    for i in mut1:
                        mut.append(i)
                        
                    ###below increases iteration count
                    c+=1
                    ### every 100 iterations check if cost is same as previous cost 100 iterations ago
                    if c %100 == 0:
                        ###if cost is same, increase count by 1
                        if costfunc(sim,egy,ogpulsars,mut) == prev:
                            count+=1
                            pbar.update(1)
                            ###once count reaches 40, i.e 4000 iterations no cost change, model stops iterating
                            if count == fin :
                                print('Sequence Generated')
                                break
                        else:
                            ###if cost is not the same, count is reset
                            count=0
                            pbar.reset()
                        #print(c, mut.count(1),costfunc(sim,egy,ogpulsars,mut),count,prev)
                        
                        prev=costfunc(sim,egy,ogpulsars,mut)
                        
                ####if single mutation caused no decrease in cost, mutate an additional 5 times
                else:
                    mut1=mutate(mut1)
                    mut1=mutate(mut1)
                    mut1=mutate(mut1)
                    mut1=mutate(mut1)
                    mut1=mutate(mut1)
                    #mut1=mutate(mut1)
                    #mut1=mutate(mut1)
                    #mut1=mutate(mut1)
                    
                    ### ###if mutated sequence has lower cost, save the new sequence as mut
                    if costfunc(sim,egy,ogpulsars,mut1) < costfunc(sim,egy,ogpulsars,mut):
                        mut = []
                        for i in mut1:
                            mut.append(i)
                            
                    ###below increases iteration count
                    c+=1
                    ### every 100 iterations check if cost is same as previous cost 100 iterations ago                    
                    if c %100 == 0:
                        ###if cost is same, increase count by 1
                        if costfunc(sim,egy,ogpulsars,mut) == prev:
                            count+=1
                            pbar.update(1)
                            ###once count reaches 40, i.e 4000 iterations no cost change, model stops iterating
                            if count == fin:
                                print('Sequence Generated')
                                break
                        ###if cost is not the same, count is reset
                        else:
                            count=0
                            pbar.reset()
                        #print(c, mut.count(1),costfunc(sim,egy,ogpulsars,mut),count,prev)
                        prev=costfunc(sim,egy,ogpulsars,mut)
    ###once finished iterating return the optimized generated binary sequence, the generated background sequences,
    ### the number of iterations, and the reference sequence used to create time reference.
    return(mut,ogpulsars,c,reference)

In [ ]:
pulsar_model()

Enter Similarity Value Between 0.0 and 1.0: 0.5
Enter Energy Value Between 0.0 and 1.0: 0.5
Enter Initial Channel Wavelength: 1400
Enter Minimum Flux To Be Considered A Pulse: 1
Enter Galactic X coordinate Of Search Area Center: 0
Enter Galactic Y coordinate Of Search Area Center: 0
Enter Galactic Z coordinate Of Search Area Center: 0
Enter Radius of Search Area in Kpc: 5
